In [1]:
import torch
from torch import nn
import data_utils
from training.MADGAN_train import MadGanTrainingPipeline
from models.MADGAN import Generator, Discriminator, AnomalyDetector
from utils import evaluation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Parameters

In [2]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)

cuda


In [3]:
model_type = "MAD-GAN"
num_generated_features = 7
seq_length = 30
seq_stride = 10

random_seed = 0
num_epochs = 2000
batch_size = 8
lr = 1e-5
wd = 5e-7
latent_dim = 500
hidden_dim = 500

# Load data

In [4]:
#train_dl, test_dl = data_utils.load_kdd99(seq_length, seq_stride, num_generated_features,seq_length,batch_size)

In [19]:
file_path = './data/Stocks/aapl.us.txt'
tscv_dl_list = data_utils.load_stock_as_crossvalidated_timeseries(file_path, seq_length, seq_stride, seq_length, batch_size)

# Model

In [11]:
pipeline = MadGanTrainingPipeline()

In [12]:
generator = Generator(
    latent_space_dim=latent_dim,
    hidden_units=hidden_dim,
    output_dim=num_generated_features)
generator.to(device=DEVICE)

Generator(
  (lstm): LSTM(500, 500, num_layers=2, batch_first=True, dropout=0.1)
  (linear): Linear(in_features=500, out_features=7, bias=True)
)

In [13]:
discriminator = Discriminator(input_dim=num_generated_features,
    hidden_units=hidden_dim,
    add_batch_mean=False)
discriminator.to(device=DEVICE)

Discriminator(
  (lstm): LSTM(7, 500, num_layers=2, batch_first=True, dropout=0.1)
  (linear): Linear(in_features=500, out_features=1, bias=True)
  (activation): Sigmoid()
)

# Loss and Optimizer

In [14]:
def loss_function(inputs, targets):
    return nn.BCELoss()(inputs, targets)

In [15]:
discriminator_optim = torch.optim.Adam(discriminator.parameters(), lr=lr, weight_decay=wd)
generator_optim = torch.optim.Adam(generator.parameters(), lr=lr, weight_decay=wd)

# Train

In [16]:
pipeline.train(seq_length, latent_dim, tscv_dl_list, discriminator, generator, discriminator_optim, generator_optim, loss_function, random_seed, num_epochs, DEVICE)

Epoch 0training:
G_loss: 0.42702000588178635, D_loss_real: 0.34436835497617724, D_loss_fake: 1.1141424238681794
Evaluation metrics: {'D_loss': 1.2404421091079711, 'G_acc': 0.9225000500679016, 'D_acc': 0.5387500286102295}
Epoch 1training:
G_loss: 0.7125278204679489, D_loss_real: 0.46514754444360734, D_loss_fake: 0.7090803533792496
Evaluation metrics: {'D_loss': 1.1584208250045775, 'G_acc': 0.3469444543123245, 'D_acc': 0.8265278220176697}
Epoch 2training:
G_loss: 0.7452032387256622, D_loss_real: 0.5289203315973282, D_loss_fake: 0.6751198410987854
Evaluation metrics: {'D_loss': 1.2674142122268677, 'G_acc': 0.5361111342906952, 'D_acc': 0.7286111354827881}
Epoch 3training:
G_loss: 0.6537447869777679, D_loss_real: 0.5826160281896591, D_loss_fake: 0.7769089579582215
Evaluation metrics: {'D_loss': 1.4885480165481568, 'G_acc': 0.7819444894790649, 'D_acc': 0.5415278077125549}
Epoch 4training:
G_loss: 0.5803750529885292, D_loss_real: 0.6797703295946121, D_loss_fake: 0.8714867293834686
Evaluation 

Epoch 38training:
G_loss: 0.6416303217411041, D_loss_real: 0.6290408313274384, D_loss_fake: 0.8140389651060105
Evaluation metrics: {'D_loss': 1.585551142692566, 'G_acc': 0.7708333611488343, 'D_acc': 0.36611113548278806}
Epoch 39training:
G_loss: 0.6074054807424545, D_loss_real: 0.7649978816509246, D_loss_fake: 0.848991185426712
Evaluation metrics: {'D_loss': 1.7131298542022706, 'G_acc': 0.7902778148651123, 'D_acc': 0.18972222805023192}
Epoch 40training:
G_loss: 0.6400477379560471, D_loss_real: 0.855095636844635, D_loss_fake: 0.8019737184047699
Evaluation metrics: {'D_loss': 1.6101828575134278, 'G_acc': 0.5422222435474395, 'D_acc': 0.27097223699092865}
Epoch 41training:
G_loss: 0.7472767323255539, D_loss_real: 0.8548291921615601, D_loss_fake: 0.6877225130796433
Evaluation metrics: {'D_loss': 1.446434497833252, 'G_acc': 0.2697222322225571, 'D_acc': 0.42291668653488157}
Epoch 42training:
G_loss: 0.8625971257686615, D_loss_real: 0.7742657274007797, D_loss_fake: 0.5906600385904313
Evaluatio

Epoch 76training:
G_loss: 0.7940711170434952, D_loss_real: 0.7297241806983947, D_loss_fake: 0.6690591692924499
Evaluation metrics: {'D_loss': 1.2938035488128663, 'G_acc': 0.34000001549720765, 'D_acc': 0.5883333563804627}
Epoch 77training:
G_loss: 0.905665522813797, D_loss_real: 0.6546000614762306, D_loss_fake: 0.5797464102506638
Evaluation metrics: {'D_loss': 1.1222514390945435, 'G_acc': 0.2219444543123245, 'D_acc': 0.7222222685813904}
Epoch 78training:
G_loss: 1.0278630077838897, D_loss_real: 0.5841785475611687, D_loss_fake: 0.52440325319767
Evaluation metrics: {'D_loss': 1.0338966131210328, 'G_acc': 0.21805556416511535, 'D_acc': 0.7675000309944153}
Epoch 79training:
G_loss: 1.0118575811386108, D_loss_real: 0.5467798724770546, D_loss_fake: 0.5346010327339172
Evaluation metrics: {'D_loss': 1.0398492574691773, 'G_acc': 0.2580555647611618, 'D_acc': 0.7508333802223206}
Epoch 80training:
G_loss: 1.0245573610067367, D_loss_real: 0.5644236892461777, D_loss_fake: 0.5514687955379486
Evaluation

Epoch 114training:
G_loss: 0.8216993570327759, D_loss_real: 0.6212136596441269, D_loss_fake: 0.604423651099205
Evaluation metrics: {'D_loss': 1.1967291116714478, 'G_acc': 0.23777779042720795, 'D_acc': 0.7820833683013916}
Epoch 115training:
G_loss: 0.8600736439228058, D_loss_real: 0.5968905165791512, D_loss_fake: 0.5817211419343948
Evaluation metrics: {'D_loss': 1.166504406929016, 'G_acc': 0.22361111938953399, 'D_acc': 0.7979166865348816}
Epoch 116training:
G_loss: 0.8755437850952148, D_loss_real: 0.5926736488938331, D_loss_fake: 0.5757800579071045
Evaluation metrics: {'D_loss': 1.1813803434371948, 'G_acc': 0.26916667222976687, 'D_acc': 0.7583333611488342}
Epoch 117training:
G_loss: 0.8626030445098877, D_loss_real: 0.6004093810915947, D_loss_fake: 0.5806614905595779
Evaluation metrics: {'D_loss': 1.1964916467666626, 'G_acc': 0.25222223103046415, 'D_acc': 0.7562500357627868}
Epoch 118training:
G_loss: 0.8114777356386185, D_loss_real: 0.6049528189003468, D_loss_fake: 0.6176824301481247
Ev

Epoch 152training:
G_loss: 1.3624310970306397, D_loss_real: 0.42267067134380343, D_loss_fake: 0.48750788271427153
Evaluation metrics: {'D_loss': 0.8236529588699341, 'G_acc': 0.24472223371267318, 'D_acc': 0.8127778053283692}
Epoch 153training:
G_loss: 1.7102740466594697, D_loss_real: 0.326811845600605, D_loss_fake: 0.43756730407476424
Evaluation metrics: {'D_loss': 0.8240154385566711, 'G_acc': 0.2597222328186035, 'D_acc': 0.8000000357627869}
Epoch 154training:
G_loss: 2.0053762078285216, D_loss_real: 0.34540331475436686, D_loss_fake: 0.3838870175182819
Evaluation metrics: {'D_loss': 0.8626400113105774, 'G_acc': 0.33805557191371916, 'D_acc': 0.791111147403717}
Epoch 155training:
G_loss: 1.8399584978818893, D_loss_real: 0.36552466824650764, D_loss_fake: 0.4660350397229195
Evaluation metrics: {'D_loss': 0.869194746017456, 'G_acc': 0.2611111253499985, 'D_acc': 0.795555591583252}
Epoch 156training:
G_loss: 1.6693381994962693, D_loss_real: 0.4962060540914536, D_loss_fake: 0.4775072053074837
E

...

Epoch 1133training:
G_loss: 0.8034362316131591, D_loss_real: 0.6029690206050873, D_loss_fake: 0.6078342586755753
Evaluation metrics: {'D_loss': 1.1794007778167725, 'G_acc': 0.2383333444595337, 'D_acc': 0.8481944918632507}
Epoch 1134training:
G_loss: 0.8095537960529328, D_loss_real: 0.5628689482808114, D_loss_fake: 0.6080342918634415
Evaluation metrics: {'D_loss': 1.1665156126022338, 'G_acc': 0.29277779459953307, 'D_acc': 0.8073611378669738}
Epoch 1135training:
G_loss: 0.7928114533424377, D_loss_real: 0.5643033772706986, D_loss_fake: 0.6245948016643524
Evaluation metrics: {'D_loss': 1.2303396940231324, 'G_acc': 0.40305557250976565, 'D_acc': 0.719444465637207}
Epoch 1136training:
G_loss: 0.7474556684494018, D_loss_real: 0.6143477857112885, D_loss_fake: 0.6697420924901962
Evaluation metrics: {'D_loss': 1.3630942344665526, 'G_acc': 0.5608333468437194, 'D_acc': 0.5555555701255799}
Epoch 1137training:
G_loss: 0.6732377648353577, D_loss_real: 0.703579968214035, D_loss_fake: 0.7411049842834473

Epoch 1170training:
G_loss: 0.7221846222877503, D_loss_real: 0.7311066180467606, D_loss_fake: 0.672426187992096
Evaluation metrics: {'D_loss': 1.3522150754928588, 'G_acc': 0.32638890743255616, 'D_acc': 0.585972249507904}
Epoch 1171training:
G_loss: 0.7441400825977326, D_loss_real: 0.6850362360477448, D_loss_fake: 0.6511852949857712
Evaluation metrics: {'D_loss': 1.2914548873901368, 'G_acc': 0.2602777868509293, 'D_acc': 0.7593055844306946}
Epoch 1172training:
G_loss: 0.766922727227211, D_loss_real: 0.6463573485612869, D_loss_fake: 0.6337826251983643
Evaluation metrics: {'D_loss': 1.2548028945922851, 'G_acc': 0.25611112713813783, 'D_acc': 0.8011111378669739}
Epoch 1173training:
G_loss: 0.757521340250969, D_loss_real: 0.6230489671230316, D_loss_fake: 0.6437913060188294
Evaluation metrics: {'D_loss': 1.2790353059768678, 'G_acc': 0.3625000178813934, 'D_acc': 0.7276389241218567}
Epoch 1174training:
G_loss: 0.7259644240140914, D_loss_real: 0.6304799914360046, D_loss_fake: 0.6759185075759888
E

Epoch 1207training:
G_loss: 0.7171286970376969, D_loss_real: 0.7069000512361526, D_loss_fake: 0.6771641731262207
Evaluation metrics: {'D_loss': 1.364940094947815, 'G_acc': 0.41027780175209044, 'D_acc': 0.5762500166893005}
Epoch 1208training:
G_loss: 0.7225546598434448, D_loss_real: 0.6758188277482986, D_loss_fake: 0.6802115350961685
Evaluation metrics: {'D_loss': 1.3644891738891602, 'G_acc': 0.4191666841506958, 'D_acc': 0.570555579662323}
Epoch 1209training:
G_loss: 0.7349862366914749, D_loss_real: 0.6819298177957535, D_loss_fake: 0.6699360966682434
Evaluation metrics: {'D_loss': 1.408586573600769, 'G_acc': 0.5344444692134858, 'D_acc': 0.4979166865348816}
Epoch 1210training:
G_loss: 0.7032620400190354, D_loss_real: 0.7057952523231507, D_loss_fake: 0.7092109680175781
Evaluation metrics: {'D_loss': 1.4786351680755616, 'G_acc': 0.5477778077125549, 'D_acc': 0.4480555772781372}
Epoch 1211training:
G_loss: 0.6883652031421661, D_loss_real: 0.758663672208786, D_loss_fake: 0.7212213903665543
Ev

Epoch 1244training:
G_loss: 0.8081746935844422, D_loss_real: 0.6589494705200195, D_loss_fake: 0.6012240558862686
Evaluation metrics: {'D_loss': 1.2369267225265503, 'G_acc': 0.16777778565883636, 'D_acc': 0.8369444847106934}
Epoch 1245training:
G_loss: 0.794591274857521, D_loss_real: 0.6265671849250793, D_loss_fake: 0.61326405107975
Evaluation metrics: {'D_loss': 1.244571566581726, 'G_acc': 0.274444454908371, 'D_acc': 0.7925000548362732}
Epoch 1246training:
G_loss: 0.7583869636058808, D_loss_real: 0.6173932373523712, D_loss_fake: 0.6459772557020187
Evaluation metrics: {'D_loss': 1.2959243535995484, 'G_acc': 0.4158333599567413, 'D_acc': 0.6884722590446473}
Epoch 1247training:
G_loss: 0.7102575838565827, D_loss_real: 0.629022940993309, D_loss_fake: 0.6909790396690368
Evaluation metrics: {'D_loss': 1.3555163145065308, 'G_acc': 0.6091666936874389, 'D_acc': 0.5606944739818573}
Epoch 1248training:
G_loss: 0.6718231499195099, D_loss_real: 0.6489606827497483, D_loss_fake: 0.7310663133859634
Eval

KeyboardInterrupt: 